In [153]:
import numpy as np
import matplotlib.pyplot as plt

# You may change the mhealth_activity module but your algorithm must support the original version
from mhealth_activity import Recording, Trace, Activity, WatchLocation, Path

# For interactive plots, uncomment the following line
# %matplotlib widget
import os
import pandas as pd
import pickle
from tqdm import tqdm
from scipy.fft import fft, fftfreq
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from scipy.signal import find_peaks
from scipy.signal import peak_prominences
from sklearn import svm
from sklearn.metrics import mean_absolute_error,accuracy_score,precision_score,recall_score,confusion_matrix,classification_report,f1_score
from multiprocessing import Pool

In [3]:
#read data

data = pd.read_pickle('data/pickled_and_sorted_training_data.pkl.zst')


In [76]:
# for x in data['altitude']:
#     print(x.values)

test = [x.values for x in data['altitude']]
print(test)
print(test[0])

[array([401.06787109, 401.06787109, 401.06787109, ..., 450.23861323,
       450.23861314, 450.23861314]), array([450.58095942, 450.58095942, 450.58095942, ..., 407.91178428,
       407.91178428, 407.91178428]), array([408.7502594 , 408.7502594 , 407.29495239, ..., 450.72974694,
       450.72974694, 450.72974694]), array([459.9460144 , 459.9460144 , 459.9460144 , ..., 406.84563044,
       406.84563044, 406.84563044]), array([461.61029053, 461.61029053, 461.61029053, ..., 496.6081543 ,
       496.6081543 , 496.6081543 ]), array([403.46264648, 403.46264648, 403.46264648, ..., 451.20713487,
       451.20713487, 451.20713487]), array([452.00967789, 452.00967789, 452.0096817 , ..., 407.40448512,
       407.40448512, 407.40448512]), array([454.3999939 , 454.3999939 , 454.3999939 , ..., 507.79998779,
       507.79998779, 507.79998779]), array([456.29998779, 456.29998779, 456.29998779, ..., 502.40064579,
       502.40064579, 502.40064579]), array([453.10933685, 453.10933685, 453.10933685, ..., 

In [188]:
# def features_extraction_watchpos(df, prefix): 
    
#     FEATURES = ['MIN','MAX','MEAN','RMS','VAR','STD','POWER','PEAK','P2P','CREST FACTOR','SKEW','KURTOSIS',
#             'MAX_f','SUM_f','MEAN_f','VAR_f','PEAK_f','SKEW_f','KURTOSIS_f']

#     for idx, feature in enumerate(FEATURES):
#         FEATURES[idx] = FEATURES[idx] + '_' + prefix
    
#     print(FEATURES)
#     Min=[];Max=[];Mean=[];Rms=[];Var=[];Std=[];Power=[];Peak=[];Skew=[];Kurtosis=[];P2p=[];CrestFactor=[];
#     FormFactor=[]; PulseIndicator=[];
#     Max_f=[];Sum_f=[];Mean_f=[];Var_f=[];Peak_f=[];Skew_f=[];Kurtosis_f=[]
    
#     X = df.values
#     ## TIME DOMAIN ##
#     #list of lists of lists, ugly as fuck but it works 
#     for recording in X:
#         Min.append(np.min(recording[0]))
#         Max.append(np.max(recording[0]))
#         Mean.append(np.mean(recording[0]))
#         Rms.append(np.sqrt(np.mean(recording[0]**2)))
#         Var.append(np.var(recording[0]))
#         Std.append(np.std(recording[0]))
#         Power.append(np.mean(recording[0]**2))
#         Peak.append(np.max(np.abs(recording[0])))
#         P2p.append(np.ptp(recording[0]))
#         CrestFactor.append(np.max(np.abs(recording[0]))/np.sqrt(np.mean(recording[0]**2)))
#         Skew.append(stats.skew(recording[0]))
#         Kurtosis.append(stats.kurtosis(recording[0]))
#         FormFactor.append(np.sqrt(np.mean(recording[0]**2))/np.mean(recording[0]))
#         PulseIndicator.append(np.max(np.abs(recording[0]))/np.mean(recording[0]))
#         ## FREQ DOMAIN ##
#         ft = fft(recording[0])
#         S = np.abs(ft**2)/len(df)
#         Max_f.append(np.max(S))
#         Sum_f.append(np.sum(S))
#         Mean_f.append(np.mean(S))
#         Var_f.append(np.var(S))
        
#         Peak_f.append(np.max(np.abs(S)))
#         Skew_f.append(stats.skew(recording[0]))
#         Kurtosis_f.append(stats.kurtosis(recording[0]))

    
#     #Create dataframe from features
#     df_features = pd.DataFrame(index = [FEATURES], 
#                                data = [Min,Max,Mean,Rms,Var,Std,Power,Peak,P2p,CrestFactor,Skew,Kurtosis,
#                                        Max_f,Sum_f,Mean_f,Var_f,Peak_f,Skew_f,Kurtosis_f])
#     return df_features

def features_extraction_common(inp): 
    df, prefix = inp
    FEATURES = ['Median', 'Numneg', 'Numpos', 'Numabovmed', 'Mean', 'STD', 'MAD', 'Var', 'Min', 'Max', 'SMA', 'Energy', 'IQR', 'Entropy', 'Npeaks', 'avgprom', 'avgpeakdist', 'Sum_f', 'Max_f', 'NPeak_f', 'Avgprom_f','avgpeakdist_f', 'Mean_f', 'Skew_f', 'Kurtosis_f']

    for idx, feature in enumerate(FEATURES):
        FEATURES[idx] = FEATURES[idx] + '_' + prefix
    
    print(FEATURES)
    Median=[];Numneg=[];Numpos=[];Numabovmed=[];
    Min=[];Max=[];Mean=[];Mad=[];Sma=[];Eng=[];Iqr=[];Entr=[];Std=[];Var=[];Kurt=[];Skew=[];Npeaks=[];Avgprom=[]
    Min_d=[];Max_d=[];Mean_d=[];Mad_d=[];Sma_d=[];Eng_d=[];Iqr_d=[];Entr_d=[];Std_d=[];Var_d=[];
    Max_f=[];NPeak_f=[];Avgprom_f=[];Mean_f=[];Skew_f=[];Kurtosis_f=[];Sum_f=[]; Avgpeakdist=[]; Avgpeakdist_f=[];
    
    X = df.values
    ## TIME DOMAIN ##
    #list of lists of lists, ugly as fuck but it works 
    for recording in X:
        Median.append(np.median(recording[0]))
        Numneg.append(np.sum(np.array(recording[0]) < 0, axis=0))
        Numpos.append(np.sum(np.array(recording[0]) > 0, axis=0))
        Numabovmed.append(np.sum(np.array(recording[0]) > np.median(recording[0]), axis=0))

        Mean.append(np.mean(recording[0]))
        Std.append(np.std(recording[0]))
        #median absolute deviation
        Mad.append(stats.median_abs_deviation(recording[0], scale=1))
        Var.append(np.var(recording[0]))
        Min.append(np.min(recording[0]))
        Max.append(np.max(recording[0]))
        #Signal Magnitude Area
        Sma.append(np.sum(recording[0]))
        #energy measure
        Eng.append(np.sum(recording[0]**2)/len(recording[0]))
        Iqr.append(stats.iqr(recording[0]))
        Entr.append(stats.entropy(recording[0]))

        npeaks, _ = find_peaks(recording[0], distance=5)
        Npeaks.append(len(npeaks))
        prom = peak_prominences(recording[0], npeaks)
        Avgprom.append(np.mean(prom))

        Apeakdist = 0
        for i in range(len(npeaks)-1):
            Apeakdist += abs(npeaks[i] - npeaks[i+1])
        Avgpeakdist.append(Apeakdist/(len(npeaks)-1))

        ## FREQ DOMAIN ##
        ft = np.abs(fft(recording[0]))
        Sum_f.append(np.sum(ft))
        Max_f.append(np.max(ft))

        npeaks, _ = find_peaks(ft, distance=5)
        NPeak_f.append(len(npeaks))
        prom = peak_prominences(ft, npeaks)
        Avgprom_f.append(np.mean(prom))

        Apeakdist = 0
        for i in range(len(npeaks)-1):
            Apeakdist += abs(npeaks[i] - npeaks[i+1])
        Avgpeakdist_f.append(Apeakdist/(len(npeaks)-1))

        Mean_f.append(np.mean(ft))
        Skew_f.append(stats.skew(ft))
        Kurtosis_f.append(stats.kurtosis(ft))

        #derivative
        # f = np.gradient(recording[0])
        # Mean_d.append(np.mean(recording[0]))
        # Std_d.append(np.std(recording[0]))
        # #median absolute deviation
        # Mad_d.append(stats.median_abs_deviation(recording[0], scale=1))
        # Var_d.append(np.var(recording[0]))
        # Min_d.append(np.min(recording[0]))
        # Max_d.append(np.max(recording[0]))
        # #Signal Magnitude Area
        # Sma_d.append(np.sum(recording[0]))
        # #energy measure
        # Eng_d.append(np.sum(recording[0]**2)/len(recording[0]))
        # Iqr_d.append(stats.iqr(recording[0]))
        # Entr_d.append(stats.entropy(recording[0]))

    #Create dataframe from features
    df_features = pd.DataFrame(index = [FEATURES], 
                               data = [Median, Numneg, Numpos, Numabovmed, Mean,Std, Mad, Var, Min, Max, Sma, Eng, Iqr, Entr, Npeaks, Avgprom, Avgpeakdist, Sum_f, Max_f, NPeak_f, Avgprom_f, Avgpeakdist_f, Mean_f, Skew_f, Kurtosis_f]) 
    df_features = pd.DataFrame.transpose(df_features)
    df_features.columns = df_features.columns.map(''.join)
    # get rid of multiindex
    return df_features

# def features_extraction_path(df, prefix): 



In [5]:
#load pickled training 3d norm accelerometer data
file = open('data/accel_mag_train.pkl', 'rb')
pickled = pickle.load(file)
accel_mag_train =  pd.DataFrame(((x,) for x in pickled), columns=['lists'])
print(accel_mag_train.shape)
file.close()

file = open('data/magneto_mag_train.pkl', 'rb')
pickled = pickle.load(file)
magneto_mag_train =  pd.DataFrame(((x,) for x in pickled), columns=['lists'])
print(magneto_mag_train.shape)
file.close()

file = open('data/gyro_mag_train.pkl', 'rb')
pickled = pickle.load(file)
gyro_mag_train =  pd.DataFrame(((x,) for x in pickled), columns=['lists'])
print(gyro_mag_train.shape)
file.close()

pos_labels  = []
path_labels = []
for label in data["labels"]:
    path_labels.extend([label["path_idx"]])
    pos_labels.extend([label["watch_loc"]])


temp_train =  pd.DataFrame(((x.values,) for x in data['temperature']), columns=['temp'])

(396, 1)
(396, 1)
(396, 1)


In [187]:
print(pos_labels.count(0))
print(pos_labels.count(1))
print(pos_labels.count(2))

136
123
137


In [193]:
#extract features from accelerometer norm

# features_acc = (features_extraction_common((accel_mag_train, "acc")))
# features_mag = (features_extraction_common((magneto_mag_train, "mag")))
# features_gyro = (features_extraction_common((gyro_mag_train, "gyro")))
# features_temp = (features_extraction_common((temp_train, "temp")))

features_to_calculate = ((accel_mag_train, "acc"), (magneto_mag_train, "mag"), (gyro_mag_train, "gyro"), (temp_train, "temp"))

with Pool(4) as p:
    feat = p.map(features_extraction_common, features_to_calculate)

features_acc = feat[0]
features_mag = feat[1]
features_gyro = feat[2]
features_temp = feat[3]

features_acc 



In [190]:
#acc + mag yield the best results(87%), acc + mag + gyro 85%, mag + gyro 85%, acc + gyro  82.5%, 

features = pd.concat([features_acc, features_mag, features_gyro, features_temp], axis=1)


In [191]:
#split the training data because test data doesnt have labels
#why help the students even slightly?

pos_train_features, pos_test_features, pos_train_labels, pos_test_labels = train_test_split(
features, pos_labels, test_size=0.2, random_state=10)

scaler = StandardScaler()
X_train_pos = scaler.fit_transform(pos_train_features) 

# Instantiate model with 1000 decision trees, use all cores
rf_pos = RandomForestRegressor(n_estimators = 1000, random_state = 42, n_jobs=-1)
# Train the model on training data
rf_pos.fit(X_train_pos, pos_train_labels)


RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=42)

In [192]:
def evaluate_pos(model, test_features, test_labels):
    
    y_pred = model.predict(test_features)

    #cast to int because indices cant be floats
    watch_pos_pred = np.rint(y_pred)

    num_tests = len(watch_pos_pred)

    watch_pos_mismatches = np.count_nonzero(watch_pos_pred != test_labels)

    print(f"{num_tests} tests, {watch_pos_mismatches} watch pos mismatches {(1-(watch_pos_mismatches/num_tests))*100} accuracy")
    return (1-(watch_pos_mismatches/num_tests))*100

scaler = StandardScaler()
X_val = scaler.fit_transform(pos_test_features) 


evaluate_pos(rf_pos,X_val,pos_test_labels)




80 tests, 14 watch pos mismatches 82.5 accuracy


82.5

In [180]:
def test_sensor_combinations():
    for i in range(1,16):
        feat = []
        print(i)
        print("[", end='')
        if(i&1):
            feat.append(features_acc)
            print("acc, ", end='')
        if(i&(1<<1)):
            feat.append(features_mag)
            print("mag, ", end='')
        if(i&(1<<2)):
            feat.append(features_gyro)
            print("gyro, ", end='')
        if(i&(1<<3)):
            feat.append(features_temp)
            print("temp, ", end='')
        print("]")
        features = pd.concat(feat, axis=1)

        pos_train_features, pos_test_features, pos_train_labels, pos_test_labels = train_test_split(
        features, pos_labels, test_size=0.2, random_state=10)

        scaler = StandardScaler()
        X_train_pos = scaler.fit_transform(pos_train_features) 

        # Instantiate model with 1000 decision trees, use all cores
        rf_pos = RandomForestRegressor(n_estimators = 1000, random_state = 42, n_jobs=-1)
        # Train the model on training data
        rf_pos.fit(X_train_pos, pos_train_labels)

        X_val = scaler.fit_transform(pos_test_features) 
        evaluate_pos(rf_pos,X_val,pos_test_labels)

test_sensor_combinations()

1
[acc, ]
80 tests, 15 watch pos mismatches 81.25 accuracy
2
[mag, ]
80 tests, 53 watch pos mismatches 33.75 accuracy
3
[acc, mag, ]
80 tests, 15 watch pos mismatches 81.25 accuracy
4
[gyro, ]
80 tests, 15 watch pos mismatches 81.25 accuracy
5
[acc, gyro, ]
80 tests, 15 watch pos mismatches 81.25 accuracy
6
[mag, gyro, ]
80 tests, 15 watch pos mismatches 81.25 accuracy
7
[acc, mag, gyro, ]
80 tests, 15 watch pos mismatches 81.25 accuracy
8
[temp, ]
80 tests, 54 watch pos mismatches 32.49999999999999 accuracy
9
[acc, temp, ]
80 tests, 16 watch pos mismatches 80.0 accuracy
10
[mag, temp, ]
80 tests, 52 watch pos mismatches 35.0 accuracy
11
[acc, mag, temp, ]
80 tests, 15 watch pos mismatches 81.25 accuracy
12
[gyro, temp, ]
80 tests, 16 watch pos mismatches 80.0 accuracy
13
[acc, gyro, temp, ]
80 tests, 16 watch pos mismatches 80.0 accuracy
14
[mag, gyro, temp, ]
80 tests, 15 watch pos mismatches 81.25 accuracy
15
[acc, mag, gyro, temp, ]
80 tests, 15 watch pos mismatches 81.25 accuracy
